# Configuration

## Imports

In [1]:
import pandas as pd
import glob
import os
import gc
from pyspark.sql import SparkSession
from tkinter import Tk     
from tkinter.filedialog import askopenfilename,askdirectory

## Spark Configuration

In [2]:
spark = SparkSession \
    .builder \
    .appName("Data_Forecasting") \
    .config("spark.memory.offHeap.enabled","true") \
    .config("spark.driver.cores","4") \
    .config("spark.driver.maxResultSize","4g") \
    .config("spark.memory.offHeap.size","4g") \
    .config("spark.driver.memory", "4g")\
    .config("spark.executor.memory","4g")\
    .getOrCreate()

23/07/06 15:33:02 WARN Utils: Your hostname, nuno-g14 resolves to a loopback address: 127.0.1.1; using 192.168.1.103 instead (on interface wlp2s0)
23/07/06 15:33:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/06 15:33:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/06 15:33:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
Tk().withdraw()

while True:
    file_or_folder_input = input("File (f) or Directory (d):")
    if file_or_folder_input == 'f':
        path = askopenfilename()
        separator = input("CSV Separator (: or ; or , or .):")
        dataframe = spark.read.load(path,format="csv", sep=separator, inferSchema="true", header="true")
        break
    elif file_or_folder_input == 'd':
        path = askdirectory()
        separator = input("CSV Separator (: or ; or , or .):")
        
        li = []
        files = os.path.join(path, "*.csv")
        files = glob.glob(files)

        for filename in files:
            dataframe = spark.read.csv(filename, header='true',sep=separator)
            li.append(dataframe)
            
        df0 = li[0]
        join_or_union = input("Join (j) or Union (u):")
        
        if join_or_union == 'j':
            for df in li:
                if df0 != df:
                    df0 = df0.join(df,["Date"], "outer")
        elif join_or_union == 'u':
            for df in li:
                df0 = df0.union(df)
        else:
            print('Incorrect input')

        dataframe = df0
        
        break
    else:
        print('Incorrect input')

File (f) or Directory (d): d


KeyboardInterrupt: Interrupted by user

In [ ]:
dataframe.show(5)

In [13]:
dataframe.summary().show()

23/04/22 21:10:15 WARN DAGScheduler: Broadcasting large task binary with size 1933.6 KiB
+-------+-------+-----------------+-----------------+------------------+------------------+------------------+------------------+--------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+--------------------+------------------+--------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+-----------------+---------------------+----

In [ ]:
write_path = input("Name of the new parquet file (no need to input .parquet):")
dataframe.write.parquet("../MSc_Model_Datasets/" + write_path + ".parquet")